# Object Detection and Image Cropping

This notebook will use [Object Detection model from TensorFlow Hub](https://www.tensorflow.org/hub/tutorials/object_detection)

## Setup


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

In [2]:
image_path = 'image.jpg'

## Object Detection

**FasterRCNN+InceptionResNet V2**

In [3]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

detector = hub.load(module_handle).signatures['default']

In [4]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [5]:
def run_detector(detector, path):
  img = load_img(path)
  h, w = img.shape[0], img.shape[1]

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  result = detector(converted_img)

  result = {key:value.numpy() for key,value in result.items()}

  crop = {'score':[], 'boxes':[]}

  for i,entity in enumerate(result['detection_class_entities']):
    if entity == b'Person' and result['detection_scores'][i] > 0.4:
      crop['score'].append(result['detection_scores'][i])
      crop['boxes'].append(result['detection_boxes'][i])

  if not crop.get('score'):
      return [0, 0, 1, 1], h, w

  return crop['boxes'][np.argmax(crop['score'])], h, w

## Image Cropping

In [6]:
def crop_image(detector, path):
  boxes, h, w = run_detector(detector, path)
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  extension = path.split('.')[-1]
  cropped_image = image[int(boxes[0] * (h-1)):(int(boxes[2] * (h-1)))+1, int(boxes[1] * (w-1)):(int(boxes[3] * (w-1)))+1]
  cv2.imwrite(f'result.{extension}', cropped_image)

In [7]:
crop_image(detector, image_path)